Shap används ungefär som permutation importance för att hitta bästa features för en LSTM med tidsseriedata

In [ ]:
def predict_fn(x):
    # Convert the input to tensor if it's not and ensure it has the right shape
    if not isinstance(x, torch.Tensor):
        x = torch.tensor(x, dtype=torch.float32)
    
    # Ensure x is 3D: [batch_size, sequence_length, features]
    if x.dim() == 2:
        x = x.unsqueeze(1)  # Adds a sequence length of 1
    
    # Move the tensor to the correct device
    x = x.to(device)
    
    # Ensure model is in evaluation mode
    model.eval()
    
    with torch.no_grad():
        predictions = model(x)
        # Convert predictions to probabilities if it's a classification model
        predictions = torch.softmax(predictions, dim=1).cpu().numpy()
    
    #return predictions.cpu().numpy()
    return predictions

In [ ]:
import shap

# Sample a subset of your validation dataset and ensure it's 2D
# Here, `shap.sample()` is replaced with manual sampling and reshaping for clarity
num_samples = 100  # Number of samples to use for background distribution
selected_time_step = 0  # Example: choose the first time step for simplicity

# Ensure X_val_np is a numpy array and select a specific time step for all samples
X_val_sampled = X_val_np[:num_samples, selected_time_step, :]  # This selects the first time step for the first `num_samples`

# Now, ensure that explainer is created with 2D data
explainer = shap.KernelExplainer(predict_fn, X_val_sampled)

# Similarly, when computing SHAP values, ensure the input data is 2D
X_val_to_explain = X_val_np[:10, selected_time_step, :]  # Explain the first 10 samples at the selected time step
shap_values = explainer.shap_values(X_val_to_explain)


shap.initjs()
# Visualization
shap.force_plot(
    base_value=explainer.expected_value[0],  # Base value or expected value of the model
    shap_values=shap_values[0],  # SHAP values for a single instance or batch
    features=X_val_to_explain  # Features corresponding to shap_values
)


In [ ]:
shap.interaction_plot(feature_index, shap_values, features)

# If shap_values is a list of lists with a single inner list (for a single-output model),
# extract the inner list:
if isinstance(shap_values, list) and len(shap_values) == 1:
    shap_values = shap_values[0]

# Now try creating the beeswarm plot again
shap.summary_plot(shap_values, X_val_np[0], plot_type="beeswarm")